In [ ]:
import pandas as pd

In [ ]:
pip install nltk

In [ ]:
import nltk
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [ ]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
train_df=pd.read_csv("/content/drive/MyDrive/Fake_News_ShareTask/Task1/Fake_train.csv")
train_df.head(5)

,text,label
0,നല്ല അവതരണം. സത്യം പുറത്തു വരട്ടെ,Fake
1,Masha Allah,Fake
2,"അന്വേഷണം കഴിയുമ്പോൾ,. C. A. A. യ്ക്ക് എതിരായ ക...",Fake
3,Illathentha avaru purath vidayittalland verenth,Fake
4,Barana pakshathin matoru niyamam.nalla moyanth...,original


In [ ]:
train_df.shape

(3257, 2)

In [ ]:
val_df=pd.read_csv("/content/drive/MyDrive/Fake_News_ShareTask/Task1/Fake_dev.csv")
val_df.head(5)

,text,label
0,Full. Musilm. Verodamum,Fake
1,പക്ഷികളും മൃഗങ്ങളും ഈ ലോകത്ത് സുഖമായി ജീവിക്കു...,Fake
2,ഒരു താടിക്കാരൻ പാത്രം കൊട്ടാൻ പറഞ്ഞപ്പോ .........,original
3,കുംഭളേമ<br>മറന്നോ,original
4,ഇത് തിരുവാതിര അല്ല...... കോറോണയെ കൈകൊട്ടി കൊല്...,original


In [ ]:
val_df.shape

(815, 2)

In [ ]:
test_df=pd.read_csv("/content/drive/MyDrive/Fake_News_ShareTask/Task1/Fake_test_without_labels.csv")
test_df.head(5)

,Id,text
0,Fake_01,5000 ഉള്ള പോൾ ലോഗ്‌ഡ്‌വൻ ഇപ്പോള് 250000 എന്താ...
1,Fake_02,ഓഷോ രജനീഷ് പറഞ്ഞപോലെ എനിക്കപ്പോൾ തോന്നിയത് അ...
2,Fake_03,ചേട്ടാ വാർത്ത വയ്ക്കുന്നത് കേരളത്തിലാണ് സം...
3,Fake_04,Shame for entire Woman&#39;s of Kerala
4,Fake_05,135 code janaghal andhu wide business cheythal...


In [ ]:
test_df.shape

(1019, 2)

In [ ]:
# Download necessary NLTK resources
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
nltk.download('averaged_perceptron_tagger_eng')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger_eng.zip.


True

In [ ]:
import re
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer, WordNetLemmatizer
from nltk.corpus import wordnet

# Custom stopwords derived from the Malayalam dataset
custom_stopwords = {
    'അത്', 'ആണ്', 'ഒരു', 'എന്ന', 'കൂടെ', 'വീട്', 'അല്ല', 'എല്ലാം', 'ഇത്', 'പിന്നെ', 'ആരും',
    'പോയി', 'ഇവിടെ', 'അവിടെ', 'നല്ല', 'മാത്രം', 'അവന്', 'ആരും', 'നോക്കൂ', 'നീ', 'അവള്',
    'എവിടെ', 'നിനക്ക്', 'വളരെ', 'വന്ന', 'എങ്ങനെ', 'നിന്റെ', 'കാണാൻ', 'ഇവന്റെ', 'പോലെ'
}

# Function to get POS tag for lemmatization
def get_wordnet_pos(word):
    """Map POS tag to first character lemmatize() accepts."""
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {
        'J': wordnet.ADJ,
        'N': wordnet.NOUN,
        'V': wordnet.VERB,
        'R': wordnet.ADV
    }
    return tag_dict.get(tag, wordnet.NOUN)

# Preprocessing function
def preprocess_text(text, stopwords):
    """Preprocess text by cleaning, tokenizing, removing stopwords, and stemming/lemmatization."""
    stemmer = PorterStemmer()
    lemmatizer = WordNetLemmatizer()

    if isinstance(text, str):
        # Lowercase conversion
        text = text.lower()

        # Remove URLs
        text = re.sub(r'http\S+|www\S+', '', text)

        # Remove HTML tags
        text = re.sub(r'<.*?>', '', text)

        # Remove special characters and numbers
        text = re.sub(r'[^a-zA-Z\u0D00-\u0D7F\s]', '', text)  # Malayalam Unicode range

        # Remove punctuation
        text = re.sub(r'[^\w\s]', '', text)

        # Tokenize
        tokens = word_tokenize(text)

        # Remove custom stopwords
        tokens = [word for word in tokens if word not in stopwords]

        # Remove short words
        tokens = [word for word in tokens if len(word) > 2]

        # Lemmatization
        tokens = [lemmatizer.lemmatize(word, get_wordnet_pos(word)) for word in tokens]

        # Stemming
        tokens = [stemmer.stem(word) for word in tokens]

        # Remove duplicate words
        tokens = list(dict.fromkeys(tokens))

        return ' '.join(tokens)
    return text

# Replace 'Text' column with preprocessed data
train_df['text'] = train_df['text'].apply(lambda x: preprocess_text(x, custom_stopwords))
val_df['text'] = val_df['text'].apply(lambda x: preprocess_text(x, custom_stopwords))
test_df['text'] = test_df['text'].apply(lambda x: preprocess_text(x, custom_stopwords))

In [ ]:
# Save processed datasets
train_df.to_csv('/content/drive/MyDrive/Fake_News_ShareTask/Task1/Preprocessing/processed_train.csv', index=False)
val_df.to_csv('/content/drive/MyDrive/Fake_News_ShareTask/Task1/Preprocessing/processed_dev.csv', index=False)
test_df.to_csv('/content/drive/MyDrive/Fake_News_ShareTask/Task1/Preprocessing/processed_test.csv', index=False)

print("Preprocessing complete. Cleaned datasets saved.")

Preprocessing complete. Cleaned datasets saved.
